In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import re
import nltk
from matplotlib import pyplot as plt

In [2]:
dataset = pd.read_csv('news_summary.csv')#,encoding="ISO-8859-1")
dataset.head(10)

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",Virgin' now corrected to 'Unmarried' in IGIMS'...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...
5,Sonu Kumari,"03 Aug 2017,Thursday","Man found dead at Delhi police station, kin al...",http://www.hindustantimes.com/delhi-news/man-f...,A 32-year-old man on Wednesday was found hangi...,An alleged suspect in a kidnapping case was fo...
6,Parmeet Kaur,"03 Aug 2017,Thursday",Delhi HC reduces aid for 'negligent' accident ...,http://indiatoday.intoday.in/story/delhi-high-...,The Delhi High Court reduced the compensation ...,"In an interesting ruling, the Delhi high court..."
7,Chhavi Tyagi,"03 Aug 2017,Thursday",60-yr-old lynched over rumours she was cutting...,http://www.hindustantimes.com/india-news/60-ye...,A 60-year old Dalit woman was allegedly lynche...,A 60-year old Dalit woman was allegedly lynch...
8,Parmeet Kaur,"03 Aug 2017,Thursday",Chopper flying critically low led to 2015 Bomb...,http://www.hindustantimes.com/mumbai-news/2015...,An inquiry by the Aircraft Accident Investigat...,Two years after a helicopter crash near the Bo...
9,Sumedha Sehra,"03 Aug 2017,Thursday",Congress opens 'State Bank of Tomato' in Lucknow,http://indiatoday.intoday.in/story/lucknow-sta...,The Congress party has opened a bank called 'S...,"It sounds like satire, but make no mistake: at..."


In [3]:
dataset = dataset.dropna(how='any',axis=0) 
x = dataset['ctext']
y = dataset['text']
# x.datatype
# print(x[0])
t=[]
max_length_target,max_length_input = max(x),max(y)
# print(len(x[0]))
# for i in range(len(x)):
#     for j in range(len(x[i])):
#         t.append(x[j].split("."))
# print(t)
# list_of_words=[]
# t.append(x[0].split("."))
# list_of_words.append(t[0][0].split(" "))
# print(list_of_words)

# print(len(t[0]))
# for i in range(len(t[0])):
#     t.append(t[0][i])
# print(t)


# s= pd.Series([x[0],np.nan])
# print(s)
# li =[]
# s.str.split(pat=".")
# print(s[0])
# for i in x[0]:
#     c=0
#     if(i != '.'):
#         c+=1
#     else:
#         li.append()
        
headline = dataset['headlines']
hardik = dataset['headlines']
hardik_text = dataset['text']
# print(headline)
del dataset
# headline

In [4]:
def preprocess_sentence(w):
    w = w.lower().strip()

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> '+ w + ' <end>'
    return w

In [5]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, lang_tokenizer

In [6]:
# f=0
x = [preprocess_sentence(w) for w in x]
y = [preprocess_sentence(w) for w in y]
headline = [preprocess_sentence(w) for w in headline]
x_train, inp_lang = tokenize(x)
y_train, targ_lang = tokenize(y)
headline_train , headline_lang = tokenize(headline)
print("Input Tensor Shape :" , x_train.shape , "Target Tensor Shape :" , y_train.shape)
print("Preprocessed Text : \n" , x[0])
# embeddings = embed(x[0])
print("Tokenized Text INP : \n" , x_train[0])
print("Tokenized Text OUT : \n" , y_train[0])

print("Headline Text : \n" , headline[0])
print("Headline Tokenized : \n" , headline_train[0])

Input Tensor Shape : (4395, 5791) Target Tensor Shape : (4395, 86)
Preprocessed Text : 
 <start> the daman and diu administration on wednesday withdrew a circular that asked women staff to tie rakhis on male colleagues after the order triggered a backlash from employees and was ripped apart on social media . the union territory ? s administration was forced to retreat within hours of issuing the circular that made it compulsory for its staff to celebrate rakshabandhan at workplace . ? it has been decided to celebrate the festival of rakshabandhan on august . in this connection , all offices departments shall remain open and celebrate the festival collectively at a suitable time wherein all the lady staff shall tie rakhis to their colleagues , ? the order , issued on august by gurpreet singh , deputy secretary personnel , had said . to ensure that no one skipped office , an attendance report was to be sent to the government the next evening . the two notifications ? one mandating the ce

In [7]:
EMBEDDING_DIMENSION= 50
# data_directory = "/glove"
# glove_weights_file_path = "/glove/glove.6B.50d.txt"

# PAD_TOKEN = 0
word2idx= [] 
idx2word= []
weights = []
# f = open("glove_weights_file_path","r")
counts =[]
count =0
with open('glove.6B.50d.txt',encoding='utf-8') as file: 
    for index, line in enumerate(file): 
        values = line.split() # Word and weights separated by space 
        word = (values[0]) # Word is first symbol on each line 
        word_weights = np.asarray(values[1:], dtype=np.float32) # Remainder of line is weights for word 
        word2idx.append(word) # PAD is our zeroth index so shift by one 
        weights.append(word_weights)
        counts.append(index)
n_vec = index+1
# n_vec = i + 1
hidden_dim = len(line.split(' ')) - 1

vecs = np.zeros((n_vec, hidden_dim), dtype=np.float32)

with open('glove.6B.50d.txt', encoding = 'utf-8') as f:
    for i, line in enumerate(f):
        vecs[i] = np.array([float(n) for n in line.split(' ')[1:]], dtype=np.float32)

average_vec = np.mean(vecs, axis=0) #unknown vector
print("glove vctor loaded")


glove vctor loaded


In [8]:
# weights.insert(0, np.random.randn(EMBEDDING_DIMENSION))
# print(vecs)
print(word2idx[1])
print(weights[0])

,
[ 4.1800e-01  2.4968e-01 -4.1242e-01  1.2170e-01  3.4527e-01 -4.4457e-02
 -4.9688e-01 -1.7862e-01 -6.6023e-04 -6.5660e-01  2.7843e-01 -1.4767e-01
 -5.5677e-01  1.4658e-01 -9.5095e-03  1.1658e-02  1.0204e-01 -1.2792e-01
 -8.4430e-01 -1.2181e-01 -1.6801e-02 -3.3279e-01 -1.5520e-01 -2.3131e-01
 -1.9181e-01 -1.8823e+00 -7.6746e-01  9.9051e-02 -4.2125e-01 -1.9526e-01
  4.0071e+00 -1.8594e-01 -5.2287e-01 -3.1681e-01  5.9213e-04  7.4449e-03
  1.7778e-01 -1.5897e-01  1.2041e-02 -5.4223e-02 -2.9871e-01 -1.5749e-01
 -3.4758e-01 -4.5637e-02 -4.4251e-01  1.8785e-01  2.7849e-03 -1.8411e-01
 -1.1514e-01 -7.8581e-01]


In [9]:
#Uploading UNKNOWN vectors weights
# UNK_TOKEN = len(weights)

word2idx.append('UNK')
weights.append(average_vec)
print(weights[word2idx.index('UNK')])
# print(np.random.randn(EMBEDDING_DIMENSION))

[-0.12920076 -0.28866628 -0.01224866 -0.05676644 -0.20210965 -0.08389011
  0.33359843  0.16045167  0.03867431  0.17833012  0.04696583 -0.00285802
  0.29099807  0.04613704 -0.20923874 -0.06613114 -0.06822549  0.07665912
  0.3134014   0.17848536 -0.1225775  -0.09916984 -0.07495987  0.06413227
  0.14441176  0.60894334  0.17463093  0.05335403 -0.01273871  0.03474107
 -0.8123879  -0.04688699  0.20193407  0.2031118  -0.03935686  0.06967544
 -0.01553638 -0.03405238 -0.06528071  0.12250231  0.13991883 -0.17446303
 -0.08011883  0.0849521  -0.01041659 -0.13705009  0.20127155  0.10069408
  0.00653003  0.01685157]


In [10]:
# h_list =[]
# for i in range(7,len(headline[0])-5):
    
#    h_list.append(headline[0][i])
# # h_list

In [11]:
# Headline Embedding
h_features ={}
h_list=[]
h_features['headline_word_indices'] = nltk.word_tokenize(headline[1])
for word in h_features['headline_word_indices']:
#     print(h_features['headline_word_indices'])
# print(len(h_features['headline_word_indices']))
    if word in word2idx:
        h_features['headline_word_indices']=weights[word2idx.index(word)]
    else:
        h_features['headline_word_indices']=weights[word2idx.index('UNK')]
    h_list.append(h_features['headline_word_indices'])
#     print(h_features['headline_word_indices'])
# print(h_list)

# h_features =[]
# h_features = nltk.word_tokenize(headline[0])
# for word in h_features:
# #     print(h_features['headline_word_indices'])
# # print(len(h_features['headline_word_indices']))
#     if word in word2idx:
#         h_features.append([weights[word2idx.index(word)]])
#     else:
#         h_features.append([weights[word2idx.index('UNK')]])
# #     print(h_features['headline_word_indices'])

# print(len(h_features))
# print()
temp = tf.convert_to_tensor(h_list)
print(temp.shape)
temp = tf.reshape(temp,(-1,temp.shape[0],temp.shape[1]))
print(temp.shape)

(16, 50)
(1, 16, 50)


In [12]:
#document Embedding
features = {}
d_list=[]
features['word_indices'] = nltk.word_tokenize(x[1]) #embedding on the document
# features['word_indices'] = [weights[word2idx.index(word)] for word in features['word_indices']]
for word in features['word_indices']:
    if word in word2idx:
        features['word_indices'] = [weights[word2idx.index(word)]]
    else:
        features['word_indices'] =[weights[word2idx.index('UNK')]]
    d_list.append(features['word_indices'])
# print(d_list)

temp1 = tf.convert_to_tensor(d_list)
temp1 = tf.reshape(temp1,(1,temp1.shape[0],temp1.shape[2]))
print(temp1.shape)

(1, 460, 50)


In [13]:
# print(len(x[0]))

In [14]:
class Encoder(tf.keras.Model):
    '''
    The code of encoder is used for text summarization. The model needs an initial_state which is the headline of the article.
    The input is supposed to be padded sequence for initial state as well as for the input. The model cuurently implements
    GRU for the encoder sequence as there is some bug with using lstm with bidirectional. Bidirectional was also tried with 
    GRU but same bug occured.
    
    The initialization is based on docuemnt context vector paper referenced from 
    https://arxiv.org/pdf/1807.08000.pdf paper named 
    Abstractive and Extractive Text Summarization using DocumentContext Vector and Recurrent Neural Networks.
    
    However the logic is modeified. The model takes the preprocessed tokenized padded sequence as input for initials state
    and the then it is passed through the embedded vector and the size of the output vector after embedding is (batch_size , time , embedding size)
    after this the model is averaged about the time axis and gives the output as (batch_size , embedding size).
    This tensor is then passed as input to dence vector and is used to give the output size of (batch_zie , lgur_hidden_units)
    '''
    def __init__(self,units = 16,activation='tanh',recurrent_activation='sigmoid',return_state=True,return_sequence=True , vocab_size=400000 , embedding_dim=50 , batch_size=2):
        super (Encoder , self).__init__()
        self.units = units
        self.activation = activation
        self.recurrent_activation = recurrent_activation
        self.return_state = return_state
        self.return_sequence = return_sequence
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size
        self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_dim)
#         self.lstm = tf.keras.layers.LSTM(units=self.units , activation=self.activation , recurrent_activation=self.activation, return_sequences=self.return_sequence , return_state=self.return_state)
        self.gru = tf.keras.layers.GRU(units=self.units , activation=self.activation , recurrent_activation=self.activation, return_sequences=self.return_sequence , return_state=self.return_state)
#         self.bidirectional = tf.keras.layers.Bidirectional(self.gru)
        self.dense = tf.keras.layers.Dense(units=self.units , input_shape = (self.embedding_dim,))
#         print(self.gru)
#         print(self.dense)
    
    def call(self,x,headline):
#         x = self.embedding(x)
#         headline = self.embedding(headline)
        print(x.shape,headline.shape)
        headline = tf.reduce_mean(headline,axis=1)
        headline  = self.dense(headline)
        output , state = self.gru(x,initial_state=headline)
#         print(headline)
        return output , state

In [15]:
encoder = Encoder()

In [16]:
# encoder(x_train[:2] , headline_train[:2])
output, state = encoder(temp1 ,temp)
print(output.shape)
print(state.shape)

(1, 460, 50) (1, 16, 50)
(1, 460, 16)
(1, 16)


In [17]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))
#         print("Score is :",score)
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [18]:
attention_layer = BahdanauAttention(0)
attention_result, attention_weights = attention_layer(state, output)
print(attention_result.shape)
print(attention_weights.shape)

(1, 16)
(1, 460, 1)


In [19]:
class Decoder(tf.keras.Model):
    
    def __init__(self, units = 16,activation='tanh',recurrent_activation='sigmoid',return_state=True,return_sequence=True , vocab_size=400000 , embedding_dim=50 , batch_size=1):
        super(Decoder, self).__init__()
        self.units = units
        self.activation = activation
        self.recurrent_activation = recurrent_activation
        self.return_state = return_state
        self.return_sequence = return_sequence
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
#         self.gru = tf.keras.layers.GRU(self.dec_units,
#                                        return_sequences=True,
#                                        return_state=True,
#                                        recurrent_initializer='glorot_uniform')
        self.gru = tf.keras.layers.GRU(units=self.units , activation=self.activation , recurrent_activation=self.activation, return_sequences=self.return_sequence , return_state=self.return_state)

        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.units)

    def call(self, x, hidden, output):
    # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
#         x = self.embedding(x)
    
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
#         print(x.shape,context_vector.shape)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
#         print(x.shape,hidden.shape,output.shape)
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[1]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights


In [20]:
decoder = Decoder()

sample_decoder_output, _, _ = decoder(tf.random.uniform((1, 1, 50)), state, output)
print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))


Decoder output shape: (batch_size, vocab size) (16, 400000)


In [21]:
# print(features['word_indices'])

In [22]:
def evaluate():
#     attention_plot = np.zeros((max_length_target, max_length_input))

#     sentence = x[0]

#     inputs = [x[0].word_index[i] for i in sentence.split(' ')]
#     inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
#                                                            maxlen=max_length_input,
#                                                            padding='post')
    inputs = tf.convert_to_tensor(temp1)
    h = tf.convert_to_tensor(temp)
    result = ''

#     hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, h)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims(tf.expand_dims(tf.convert_to_tensor(weights[word2idx.index('start')]), 0),0)
#     print(dec_input.shape)
    for t in range(100):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
#         attention_weights = tf.reshape(attention_weights, (-1, ))
#         attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()
#         print(predicted_id)
        result += word2idx[predicted_id] + ' '

        if word2idx[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims(tf.expand_dims(tf.convert_to_tensor(weights[predicted_id]), 0),0)

    return result

In [23]:
# print(x[1])

In [24]:
def translate():
    result = evaluate()

#     print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

#     attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
#     plot_attention(attention_plot, sentence.split(' '), result.split(' '))


In [25]:
translate()

(1, 460, 50) (1, 16, 50)
Predicted translation: finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay finlay 


In [26]:
word2idx.index('start')

465